In [103]:
import os                                                              
import time   
import pickle
import sys
import numpy as np                                                     
import pandas as pd   
import seaborn as sns                                                  
import matplotlib.pyplot as plt
import math
import re
                                                                       
from sklearn.metrics import mean_absolute_error                        
from sklearn.metrics import mean_squared_error                         
from sklearn.model_selection import train_test_split                   
from sklearn.preprocessing import StandardScaler                       
from sklearn.model_selection import train_test_split

import keras                                                           
from keras.optimizers import Adam                                      
from keras.models import Model, Sequential                             
from keras.layers import Input, Dense, Dropout, merge                  
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping 

import ase                                                             
import ase.build                                                       
from ase import Atoms                                                  
from ase.atoms import Atoms                                            
from ase.io import read, write                                         
from ase.calculators.dftb import Dftb                                  
from ase.units import Hartree, mol, kcal, Bohr                         
                                                                       
from src.Calculator import src_nogrd                                                       
from src.Calculator.src_nogrd import sym_func_show    
from src.Calculator.src_nogrd import symmetry_function                                        
from src.Calculator.src_nogrd import xyzArr_generator                                 
from src.Calculator.src_nogrd import feat_scaling_func                                
from src.Calculator.src_nogrd import at_idx_map_generator
from src.Calculator.src_nogrd import at_idx_map_generator_old
from src.Calculator.store_models import write_subnet_text
                                                                                                                           
import pickle
from itertools import combinations_with_replacement as comb_replace
                                                                       
import src.Utils.DirNav                                                
from src.Utils.dftb_traj_io import read_scan_traj
import src.Utils.netBuilder
from src.Utils.netBuilder import netBuilder

In [104]:
import keras
geom_filename          = 'final_Au39.xyz' # I appended your sample structures

md_train_arr_origin    = read_scan_traj(filename=geom_filename)
md_train_arr = md_train_arr_origin.copy(deep=False).reset_index(drop=True)

In [105]:
# Maintainence: Natom is a global variable 
# Assumes that the configuration does not change the Number of atoms. 
nAtoms, xyzArr = xyzArr_generator(md_train_arr)# Calculate distance dataframe from xyz coordinates
distances = src_nogrd.distances_from_xyz(xyzArr, nAtoms)

In [106]:
struct = ase.io.read(geom_filename, format = "xyz")
SUPPORTED_ELEMENTS = ['H','O','Au']

In [107]:
at_idx_map = at_idx_map_generator(md_train_arr[0])
print(at_idx_map)

{'H': array([40, 41]), 'O': array([39]), 'Au': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38])}


In [108]:
cutoff_rad = 7 #max(distances.max(1))
Rs_array = np.linspace(0.2, 7, num=14) #np.linspace(distances.min(1), max(distances.max(1)), num=int(max(distances.max(1))-min(distances.min(1))))   # based on max and min of the distances
eta_array =  5/(np.square(0.2*Rs_array))  #((max(distances.max(1))- min(distances.min(1))))/(2*np.square(min(distances.min(1))*Rs_array))
rad_params = np.array([(Rs_array[i], eta_array[i], cutoff_rad) for i in range(len(Rs_array)) ])
# angular symmetry function parameters
cutoff_ang =  5 #math.radians(min(all_ang)) #math.radians(max(all_ang))
lambd_array = np.array([-1, 1])
zeta_array = np.array([1, 4, 16]) #zeta_array = np.array([1, 4, 16])
#zeta_array = np.array([math.radians(sum(all_ang)/len(all_ang))])
eta_ang_array = np.array([0.001, 0.01, 0.05]) #eta_ang_array = np.array([0.001, 0.01, 0.05])
#eta_ang_array = np.array([math.radians(min(all_ang))])

# Each of the element need to be parametrized for all of the list. 
angList = np.array([e1+e2 for e1, e2 in comb_replace(SUPPORTED_ELEMENTS, 2)])
#print(angList)
ang_comp = {el : angList for el in SUPPORTED_ELEMENTS}
ang_params = np.array([[eta, zeta, lambd, cutoff_ang] for eta in eta_ang_array for zeta in zeta_array for lambd in lambd_array])

In [109]:
Gparam_dict = {}
for at_type in at_idx_map.keys():
   Gparam_dict[at_type] = {}
   Gparam_dict[at_type]['rad'] = {}
   for at2_rad in at_idx_map.keys():
           Gparam_dict[at_type]['rad'][at2_rad] = rad_params

   # This Section is already designed to be genera
   Gparam_dict[at_type]['ang'] = {}
   for at23_ang in ang_comp[at_type]:
       Gparam_dict[at_type]['ang'][at23_ang] = ang_params
for at_type in Gparam_dict.keys():
   print(Gparam_dict[at_type]['rad'].keys())

dict_keys(['H', 'O', 'Au'])
dict_keys(['H', 'O', 'Au'])
dict_keys(['H', 'O', 'Au'])


In [110]:
Gfunc_data = symmetry_function(distances, at_idx_map, Gparam_dict)

c:\Users\claud\Desktop\Paris\New folder\src\Calculator\src_nogrd.py:585: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Gfunc_data = pd.Series([])
c:\Users\claud\Desktop\Paris\New folder\src\Calculator\src_nogrd.py:593: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Gfunc_data[at_type] = pd.Series([])
c:\Users\claud\Desktop\Paris\New folder\src\Calculator\src_nogrd.py:593: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Gfunc_data[at_type] = pd.Series([])
c:\Users\claud\Desktop\Paris\New folder\src\Calculator\src_nogrd.py:593: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a d

In [111]:
print(Gfunc_data['Au'][32][0])

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.34472392e-88 2.71005582e-20
 8.70032063e-04 3.06233247e+00 1.46534297e+00 9.73491836e-02
 7.54098437e-01 1.12765929e+00 1.05868253e+00 4.21053479e-01
 9.16501313e-02 2.15520822e-02 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.000000

In [112]:
# Glboal Var
n_samples = len(md_train_arr)


# Maintainence // Problem: In case we have no atom for that particular element, 
# it is difficult to give the input dimension
    
# Maintain: I assume that same amount of symmetry function is used for each atom  
# Maintain: Need a input_dim_list_generator function in the future
"""
def input_dim_list_generator(element):
    # In case of element with 0 atoms in the configuration
    nAtomsForEle = len(at_idx_map(element));
    if len nAtomsForEle == 0:
        return (nAtomsForEle, )
        """
#nSymFunc = Gfunc_data['O'][0].shape[1]
n_symm_func = Gfunc_data['O'][  at_idx_map['O'][0]].shape[1]
#input_dim_list = [(len(at_idx_map[at]) ,nSymFunc, nb_feat) for at in at_idx_map.keys()]

input_dim_list = {}
for at_type in at_idx_map:
    input_dim_list[at_type] = (n_symm_func, 1)

In [113]:
todos=np.arange(0,n_samples) #Number of structures given
Feat_train_scaled = {}
for at_type in at_idx_map.keys():
    Feat_train_scaled[at_type] = {}
    for at in at_idx_map[at_type]:
        Feat_train_scaled[at_type][at] = Gfunc_data[at_type][at][todos,]

inp_train = []
for at_type in at_idx_map.keys():

    for atA in at_idx_map[at_type]:
        inp_train.append(Feat_train_scaled[at_type][atA])

In [114]:
def build_subnets(SUPPORTED_ELEMENTS):
    elemental_subnets = {}
    for element in SUPPORTED_ELEMENTS:
        globals()[str(element)+"-subnet"]=keras.models.load_model(f"{element}-subnet.h5",compile=False)               
        subnet = Sequential()
        subnet.add(globals()[str(element)+"-subnet"]) 
        elemental_subnets[element] = subnet
    return elemental_subnets

In [115]:
def build_molecular_net(at_idx_map, elemental_subnets):
    inputs = []
    subnet_outputs = []
    atomic_nets = {}
    for type, atom_indices in at_idx_map.items():
        atomic_nets[type] = {}
        for idx in atom_indices:
            input = Input(shape=(input_dim,), dtype="float32",
                            name=f"{type}-{idx}-ele")
            inputs.append(input)
            atomic_subnet = elemental_subnets[type](input)
            subnet_outputs.append(atomic_subnet)
            #atomic_nets[type][atom_idx] = atomic_subnet
    total_output = keras.layers.add(subnet_outputs)
    molecular_net = Model(inputs, total_output)
    return molecular_net

In [116]:
def build_molecular_net(at_idx_map, elemental_subnets):
    inputs = []
    subnet_outputs = []
    atomic_nets = {}
    for type, atom_indices in at_idx_map.items():
        atomic_nets[type] = {}
        for idx in atom_indices:
            input = Input(shape=(n_symm_func,), dtype="float32",
                            name=f"{type}-{idx}-ele")
            inputs.append(input)
            atomic_subnet = elemental_subnets[type](input)
            subnet_outputs.append(atomic_subnet)
    total_output = keras.layers.add(subnet_outputs)
    molecular_net = Model(inputs, total_output)
    return molecular_net

In [117]:
subnets = build_subnets(SUPPORTED_ELEMENTS)
model = build_molecular_net(at_idx_map, subnets)

In [118]:
print(model.summary())

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
H-40-ele (InputLayer)           [(None, 150)]        0                                            
__________________________________________________________________________________________________
H-41-ele (InputLayer)           [(None, 150)]        0                                            
__________________________________________________________________________________________________
O-39-ele (InputLayer)           [(None, 150)]        0                                            
__________________________________________________________________________________________________
Au-0-ele (InputLayer)           [(None, 150)]        0                                            
______________________________________________________________________________________

In [119]:
y_pred_scaled4 = model.predict(inp_train) #Application of the model
y_pred4 = y_pred_scaled4.T[0]  # in kcal/mol unit
print(y_pred4) #kcal/mol units
#np.savetxt("4e", y_pred4, delimiter=",")

[-16.015287   -15.517038   -12.3168      -6.192355    -1.1357055
   0.16012591   4.2341356    7.597695     7.9348927    8.184764
   8.023825   -12.681801    -9.435858    -9.916773    -5.6427984
  -1.264973     5.144829     7.0918283    7.163802     8.41449
   7.7672324    7.769222   -10.977253    -4.875247     2.9944346
   1.9323243    5.370228     8.74145     10.331201     8.628399
   7.6197047    8.073081     8.067297    -9.792527     0.76831365
   0.57638645   2.5987566    3.670635     5.6213245    4.496442
   6.66967      7.9920197    8.520397     8.416741   -16.318422
 -12.13116     -9.218423    -7.7949      -3.0051541    0.6445544
   3.003044     4.593564     6.8323994    7.506595     8.458003
  -7.4132614   -3.8331878   -0.5987228    1.2737851    2.7890074
   7.082328     9.705608     7.293726     6.8002195    6.6005144
   7.9850616    1.6994888    0.48626342  -1.2281044    1.0262313
   2.5051682    4.0532017    6.3958387    5.86773      5.4837236
   7.39171      8.552468  ]


In [120]:
y_pred4*0.043 #ev units
np.savetxt("4eV", y_pred4*0.043, delimiter=",")